## Execute a WPS service

his  tutorial uses a Jupyter Notebook to create a Web Processing Service (WPS) request invoking a service deployed in the context of NextGEOSS, monitors the WPS request execution and finally retrieves the execution results

* First do the imports of the Python libraries required

In [13]:
import owslib
from owslib.wps import monitorExecution
import uuid
from owslib.wps import WebProcessingService
import sys
import os
sys.path.append('/opt/anaconda/bin/')
import lxml.etree as etree
import requests
import cioppy
ciop = cioppy.Cioppy()
import shapely
import dateutil.parser
from shapely.wkt import loads 
import pandas as pd
import geopandas as gp
from datetime import datetime, timedelta
from io import BytesIO
from zipfile import ZipFile
import matplotlib.pyplot as plt
import gdal
import numpy as np
import xml.etree.ElementTree as ET

%matplotlib inline

* Connect to the WPS server

In [2]:
wps_url = 'http://gridded-data.armines.nextgeoss.terradue.com/wps/WebProcessingService'
wps = WebProcessingService(wps_url, verbose=False, skip_caps=True)

* Do a GetCapabilities WPS request and list the process:

In [3]:
wps.getcapabilities()

* List available services

In [4]:
for index, elem in enumerate(wps.processes):
    print(index, elem.identifier)

(0, 'com.terradue.wps_oozie.process.OozieAbstractAlgorithm')


* Select the **com.terradue.wps_oozie.process.OozieAbstractAlgorithm** process and print the title and abstract after having submited a WPS DescribeProcess request

In [5]:
process_id = wps.processes[0].identifier

In [6]:
process = wps.describeprocess(process_id)

In [7]:
process.title

'McCloud Map test WPS'

In [8]:
process.abstract

'TODO'

* List the WPS process inputs:

In [9]:
for input in process.dataInputs:
    print(input.identifier)

bbox
width
height
datex
dt
count
min_tile_width
min_tile_height


* Create a Python dictionary with the inputs:

In [10]:
bbox = '5.0,46.0,6.0,45.0'
width = '1009'
height = '1013'
datex = '2453385.900000'
dt = '0.00069444'
count = '3'
min_tile_width = '-1'
min_tile_height = '-1'

In [11]:
inputs = [('bbox', bbox),
        ('width', width),
        ('height', height),
        ('datex', datex),
        ('dt', dt),
        ('count', count),
        ('min_tile_width', min_tile_width),
        ('min_tile_height', min_tile_height)]

* Submit the Execute WPS request:

In [16]:
execution = owslib.wps.WPSExecution(url=wps.url)

execution_request = execution.buildRequest(process_id, 
                                           inputs, 
                                           output = [('result_osd', False)])

execution_response = execution.submitRequest(etree.tostring(execution_request))

execution.parseResponse(execution_response)

* Monitor the request:

In [17]:
execution.statusLocation

'http://gridded-data.armines.nextgeoss.terradue.com/wps/RetrieveResultServlet?id=557bed2b-0a32-49b4-aa8d-17bcd21d9a5c'

In [18]:
monitorExecution(execution)

* Check the outcome of the processing request

In [19]:
execution.isSucceded()

True

* Search for the results produced

In [20]:
for output in execution.processOutputs:
    print(output.identifier)

result_osd


* Get the reference of output with identifier=result_osd

In [21]:
results_osd = execution.processOutputs[0].data[0]

In [22]:
print(results_osd)

<ns:Reference xmlns:ns="http://www.opengis.net/wps/1.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" href="http://gridded-data.armines.nextgeoss.terradue.com/sbws/wps/mccloud_map_wps/0000000-180901000007300-oozie-oozi-W/results/description" method="GET" mimeType="application/opensearchdescription+xml"/>


* Get the href from the previsous Reference value

In [23]:
root = ET.fromstring(results_osd)
referenceUrl = root.attrib['href']

In [24]:
referenceUrl

'http://gridded-data.armines.nextgeoss.terradue.com/sbws/wps/mccloud_map_wps/0000000-180901000007300-oozie-oozi-W/results/description'

In [25]:
search_results = ciop.search(end_point=referenceUrl,
                         params=[],
                         output_fields='identifier,enclosure',
                         model='GeoTime')

In [27]:
for index, elem in enumerate(search_results):
    print(elem['enclosure'])

http://gridded-data.armines.nextgeoss.terradue.com/data/run/mccloud_map_wps/0000000-180901000007300-oozie-oozi-W/_results/result.h5
